## Notes

4. Time difference b/w the server that this script will be hosted on and IST. (Will need to update code accordingly) {O}

In [ ]:
# CRONJOB

# IST                                                                   # UTC                                                              
# At 9:16 AM [-1]                                                       # At 3:46 AM [-1]
# every 15min 9:31 AM [-2] - 3:16 PM                                    # every 15min 4:01 AM [-2] - 9:46 AM
# At 3:31 PM [-1]                                                       # At 10:01 AM [-1]

# Candle index
# if 9:30AM<time<3:25PM:                                                # if 4:00AM<time<9:55AM:
#     i = -2                                                            #     i = -2
# else:                                                                 # else:
#     i = -1                                                            #     i = -1

## Columns to update: (strategy1)

In [ ]:
# (Assuming Bullish)
# Buy Put + Buy Future

# ----Irrespective of Status---                            

# stock_cmp                                                 
# fut_cmp
# opt_cmp
# opt_plue                                    |    (CMP - entry)
# fut_plue                                    |    (CMP - entry)
# total_plue                                  |    opt_plue + fut_plue

# -----Based on Status-----

# -> status
# -> opt_exit_price                           |    (None) if Status==ACTIVE else (opt_price at the time of T1 Hit)
# -> fut_exit_price                           |    (None) if Status==ACTIVE else (fut_price at the time of T1 Hit)

# --> opt_pl                                  |    (CMP - entry) if Status==ACTIVE else (opt_exit_price - entry)
# --> fut_pl                                  |    (CMP - entry) if Status==ACTIVE else (opt_exit_price - entry)
# --> total_pl                                |    opt_pl + fut_pl





# (Assuming Bearish)
# Buy Call + Sell Future

# ----Irrespective of Status --                            

# stock_cmp                                                 
# fut_cmp
# opt_cmp
# opt_plue                                    |    (CMP - entry)
# fut_plue                                    |    (entry - CMP)
# total_plue                                  |    opt_plue + fut_plue


# -----Based on Status ----

# -> status
# -> opt_exit_price                            |    (None) if Staus==ACTIVE else (opt_price at the time of T1 Hit)
# -> fut_exit_price                            |    (None) if Staus==ACTIVE else (fut_price at the time of T1 Hit)

# --> opt _pl                                  |    (CMP - entry) if Status==ACTIVE else (opt_exit_price - entry)
# --> ful_pl                                   |    (entry - CMP) if Status==ACTIVE else (entry - opt_exit_price)
# --> total_pl                                 |    opt_pl + fut_pl

# *The exact price of of opt1 & opt2 at the time of T1 hit cannot be obtained because the script runs once every 15 min,
# therefore, instead of the exact price of opt1/2 at the time of T1 hit, we use the high's & low's of previous
# 15 min candle and assume them as the exit price. This should not affect the P&L too much as there won't be much 
# fluctuation b/w the high/low and the exact price of opt1/2 at time of T1 hit (which will obviously be < 15 min).

## Calculating Exit Price (Strategy 1)

In [ ]:
# Bullish (Buy Put + Buy Future)
# T1      (Max profit + max loss)
# stock 10:00|----^----------------|10:15 am
# OPT   10:00|------\/-------------|10:15 pm (LOSS)
# FUT   10:00|----------^----------|10:15 pm (PROFIT)


# SL      (max profit + max loss)
# stock 10:00|----\/---------------|10:15 am
# OPT   10:00|-------^-------------|10:15 pm (PROFIT)
# FUT   10:00|----------\/---------|10:15 pm (LOSS)

# Bearish (Buy Call + Sell Future)
# T1      (Max profit + max loss)
# stock 10:00|----\/---------------|10:15 am
# OPT   10:00|------\/-------------|10:15 pm (LOSS)
# FUT   10:00|----------\/---------|10:15 pm (PROFIT)


# SL      (max profit + max loss)
# stock 10:00|----^-----------------|10:15 am
# OPT   10:00|-------^--------------|10:15 pm (PROFIT)
# FUT   10:00|----------^-----------|10:15 pm (LOSS)

In [37]:
def get_active_data(df):
    return df[df.Status.isin(['ACTIVE','T1 Hit','SL Hit','Strike 1'])]

In [38]:
def update_status(df):
    for i, row in df.iterrows():
        if dt.date.today() == dt.datetime.strptime(row['Expiry'], '%d%b%Y').date():
            df.loc[i, 'Status'] = 'Expired'

In [39]:
# For strategy1
# updates data inplace (not good coding practice)
def master(df):
    today = dt.datetime.strftime(dt.datetime.today(), format="%Y-%m-%d")
    
    for ind, row in df.iterrows():
        if row["Status"]!='Expired':
        
            if row['Type']=='Bullish':
                get_bull(ind, row, today)

            else:
                get_bear(ind, row, today)

In [31]:
# candles = [tstmp O H L C V]

In [40]:
def get_bull(ind, row, today):
    # Get data 
    hs_data = {"symbol": f"NSE:{row['Stock']}-EQ", "resolution":"15", "date_format":"1", "range_from":today, "range_to":today, "cont_flag":"1"}
    ho_data = {"symbol": f"NSE:{row['Ticker Name OPT']}", "resolution":"15", "date_format":"1", "range_from":today, "range_to":today, "cont_flag":"1"}
    hf_data = {"symbol": f"NSE:{row['Ticker Name FUT']}", "resolution":"15", "date_format":"1", "range_from":today, "range_to":today, "cont_flag":"1"}
    hs = fyers.history(hs_data)
    ho = fyers.history(ho_data)
    hf = fyers.history(hf_data)

    try:                                                                                 
        if dt.time(4, 0) < dt.datetime.now().time() < dt.time(9, 55):
            i = -2
        else:
            i = -1
        sres = hs['candles'][i]
        ores = ho['candles'][i]
        fres = hf['candles'][i]
    except:
        print(f"api response not ok for Trade ID {row['Trade ID']}")
    
    # update common columns
    df.loc[ind, 'Stock CMP'] = sres[4]
    df.loc[ind, 'Current Price OPT'] = ores[4]
    df.loc[ind, 'Current Price FUT'] = fres[4]
    df.loc[ind, 'P&L OPT (UE)'] = (ores[4] - row['Entry Price OPT']) * row['Lot Size']
    df.loc[ind, 'P&L FUT (UE)'] = (fres[4] - row['Entry Price FUT']) * row['Lot Size']
    df.loc[ind, 'Total P&L (UE)'] = df.loc[ind, 'P&L OPT (UE)'] + df.loc[ind, 'P&L FUT (UE)']
    
    # update if alert is active
    if row['Status']=='ACTIVE' or row['Status']=='Strike 1':
        evaluate_active_bull(ind, row, sres, ores, fres)

In [41]:
def evaluate_active_bull(ind, row, sres, ores, fres):
    
    # check/update status
    if sres[2] >= row['T1']:                                                       # [2] => high
        df.loc[ind, 'Status'] = 'T1 Hit'                                           # [3] => low
        df.loc[ind, 'Exit Price OPT'] = ores[3]                                    
        df.loc[ind, 'Exit Price FUT'] = fres[2]                                    

    elif (dt.datetime.fromtimestamp(sres[0]).time() == dt.time(9, 45)) and (sres[4] <= row['SL']):    # UTC time zone
        if row['Status'] == 'ACTIVE':                                             # [4] => close
            df.loc[ind, 'Status'] = 'Strike 1'                   
        else:
            df.loc[ind, 'Status'] = 'SL Hit'
            df.loc[ind, 'Exit Price OPT'] = ores[2]
            df.loc[ind, 'Exit Price FUT'] = fres[3]                                   
    
    # Calculate P&L
    if df.loc[ind, 'Status']=='ACTIVE' or df.loc[ind, 'Status']=='Strike 1':     # If T1/SL is still not hit
        df.loc[ind, 'P&L OPT'] = df.loc[ind, 'P&L OPT (UE)']
        df.loc[ind, 'P&L FUT'] = df.loc[ind, 'P&L FUT (UE)']                  

    else:                                                                # If T1/SL hit, calculate P&L based on exit_price
        df.loc[ind, 'P&L OPT'] = (df.loc[ind, 'Exit Price OPT'] - row['Entry Price OPT']) * row['Lot Size']                 
        df.loc[ind, 'P&L FUT'] = (df.loc[ind, 'Exit Price FUT'] - row['Entry Price FUT']) * row['Lot Size']

    df.loc[ind, 'Total P&L'] = df.loc[ind, 'P&L OPT'] + df.loc[ind, 'P&L FUT']

In [42]:
def get_bear(ind, row, today):
    # Get data 
    hs_data = {"symbol": f"NSE:{row['Stock']}-EQ", "resolution":"15", "date_format":"1", "range_from":today, "range_to":today, "cont_flag":"1"}
    ho_data = {"symbol": f"NSE:{row['Ticker Name OPT']}", "resolution":"15", "date_format":"1", "range_from":today, "range_to":today, "cont_flag":"1"}
    hf_data = {"symbol": f"NSE:{row['Ticker Name FUT']}", "resolution":"15", "date_format":"1", "range_from":today, "range_to":today, "cont_flag":"1"}
    hs = fyers.history(hs_data)
    ho = fyers.history(ho_data)
    hf = fyers.history(hf_data)

    try:
        if dt.time(4, 0) < dt.datetime.now().time() < dt.time(9, 55):
            i = -2
        else:
            i = -1
        sres = hs['candles'][i]
        ores = ho['candles'][i]
        fres = hf['candles'][i]
    except:
        print(f"api response not ok for Trade ID {row['Trade ID']}")
    
    # update common columns
    df.loc[ind, 'Stock CMP'] = sres[4] 
    df.loc[ind, 'Current Price OPT'] = ores[4]
    df.loc[ind, 'Current Price FUT'] = fres[4]
    df.loc[ind, 'P&L OPT (UE)'] = (ores[4] - row['Entry Price OPT']) * row['Lot Size']
    df.loc[ind, 'P&L FUT (UE)'] = (row['Entry Price FUT'] - fres[4]) * row['Lot Size']
    df.loc[ind, 'Total P&L (UE)'] = df.loc[ind, 'P&L OPT (UE)'] + df.loc[ind, 'P&L FUT (UE)']
    
    # update if alert is active
    if row['Status']=='ACTIVE' or row['Status']=='Strike 1':
        evaluate_active_bear(ind, row, sres, ores, fres)

In [43]:
def evaluate_active_bear(ind, row, sres, ores, fres):
    
    # check/update status
    if sres[3] <= row['T1']:                                       # [3] => low
        df.loc[ind, 'Status'] = 'T1 Hit'                           # [2] => high
        df.loc[ind, 'Exit Price OPT'] = ores[3]                     
        df.loc[ind, 'Exit Price FUT'] = fres[3]    
        
        
    elif (dt.datetime.fromtimestamp(sres[0]).time() == dt.time(9, 45)) and (sres[4] >= row['SL']):    # UTC time zone
        if row['Status'] == 'ACTIVE':                                             # [4] => close
            df.loc[ind, 'Status'] = 'Strike 1'                   
        else:
            df.loc[ind, 'Status'] = 'SL Hit'
            df.loc[ind, 'Exit Price OPT'] = ores[2]
            df.loc[ind, 'Exit Price FUT'] = fres[2]                    
    
    # Calculate P&L
    if df.loc[ind, 'Status']=='ACTIVE' or df.loc[ind, 'Status']=='Strike 1':     # If T1/SL is still not hit
        df.loc[ind, 'P&L OPT'] = df.loc[ind, 'P&L OPT (UE)']
        df.loc[ind, 'P&L FUT'] = df.loc[ind, 'P&L FUT (UE)']                 

    else:                                                               # If T1/SL hit, calculate P&L based on exit_price
        df.loc[ind, 'P&L OPT'] = (df.loc[ind, 'Exit Price OPT'] - row['Entry Price OPT']) * row['Lot Size']                   
        df.loc[ind, 'P&L FUT'] = (row['Entry Price FUT'] - df.loc[ind, 'Exit Price FUT']) * row['Lot Size']

    df.loc[ind, 'Total P&L'] = df.loc[ind, 'P&L OPT'] + df.loc[ind, 'P&L FUT']

## Update Google sheets

In [52]:
def push_changes(sheet, df):
    values = df.values.tolist()
    for value in values:
        try:
            cell = sheet.find(str(value[0]), in_column=0)
            a1 = cell.address
            r = cell.row
            sheet.update(f"{a1}:AA{r}", [value])
            
        except:
            print(f"error updating row with Trade ID: {value[0]}")